## Pandas QC Tools

This notebook is a reference collection of functions and tips for cleaning datasets using pandas. This is not designed to be run on a specific dataset, just provide snippets for common cleaning tasks.

This notebook serves as a reference collection of functions and tips for cleaning datasets using pandas. It’s not designed to run end-to-end on a specific dataset, but rather to provide reusable snippets for common data cleaning tasks.

### Set Up:

In [ ]:
# check environment
!conda info --envs

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import janitor

In [ ]:
df = pd.read_csv('my_dataset.csv')

# copy unaltered original dataset for reference
raw_data = df.copy()

df.info()

### Process large data sets in chunks

In [ ]:
# Set size of chunks based on available memory
chunk_size = 10000 
chunks = pd.read_csv('my_large_dataset.csv', chunksize=chunk_size)

# Example of running a cleaning function on multiple chunks
results = []

for i, chunk in enumerate(chunks):
    # apply some cleaning function to column/series
    chunk['cleaned_column'] = chunk['raw_column'].apply(column_cleaner_function)
    
    # optionally filter or aggregate cleaned series
    filtered = chunk[chunk['cleaned_column'] == 'filter_value']
    results.append(filtered)

# Combine all cleaned chunks
cleaned_df = pd.concat(results, ignore_index=True)

In [ ]:
# To write to disk instead of storing all chunks in memory
chunk_size = 10000 
chunks = pd.read_csv('my_large_dataset.csv', chunksize=chunk_size)

output_file = 'cleaned_output.csv'
first_chunk = True 

for i, chunk in enumerate(chunks):
    # apply some cleaning function to column/series
    chunk['cleaned_column'] = chunk['raw_column'].apply(column_cleaner_function)
    
    # optionally filter or aggregate cleaned series
    filtered = chunk[chunk['cleaned_column'] == 'example_value']
    
    # write to the disk, only write header for the first chunk
    filtered.to_csv(output_file, mode='a', index=False, header=first_chunk)
    first_chunk = False

### Clean up columns and indexing

In [ ]:
# janitor module's method for column name cleaning
df.clean_names()

# clean column names with regular expressions
df.columns = [
    re.sub(r'\s+', '_', re.sub(r'[^\w\s]', '', col)).strip().lower()
    for col in df.columns
]

In [ ]:
# deleting columns
to_drop = ['column_a', 'column_b']
df.drop(columns=to_drop, inplace=True)

In [ ]:
# check and set indexing column
df['identifier'].is_unique
df.set_index('identifier', inplace=True)

### Fix datatypes and categoricals

In [ ]:
# Change data types
df['num_feature'] = df['num_feature'].astype(int)
df['num_feature'] = df['num_feature'].astype('float16')
df['str_feature'] = df['str_feature'].astype(str)

df['date_col'] = pd.to_datetime(df['date_col'], errors='coerce')

# Clip and cap values in a feature
df['col'] = df['col'].clip(lower=0, upper=100)

In [ ]:
# Convert each unique category into a unique integer
df['cat_encoded'] = df['cat_uncoded'].astype('category').cat.codes

# Create a new column for each category, with 1 if the row belongs to that category, else 0
pd.get_dummies(df, columns=['col'])

### Missing Data

In [ ]:
# Visualize missing values with a heatmap
sns.heatmap(df.isnull(), cmap='viridis')
plt.title("Missing Values Heatmap")
plt.show()

In [ ]:
# Return missing data report
def check_missing_values(df):
    missing_stats = pd.DataFrame({
        'total_missing': df.isnull().sum(),
        'percent_missing': (df.isnull().sum() / len(df) * 100).round(3)
    }).sort_values('percent_missing', ascending=False)
    
    missing_stats['missing_level'] = missing_stats['percent_missing'].apply(
        lambda x: 'high' if x > 15 else ('medium' if x > 5 else 'low')
    )
    return missing_stats
    
check_missing_values(df)

In [ ]:
# Get value_counts for columns with missing data, can be messy
cols_with_na = [col for col in df.columns if df[col].isnull().any()]

for col in cols_with_na:
    print(df[col].value_counts(dropna=False))

In [ ]:
# Replace missing values with mean or other value
df['col'] = df['col'].fillna(df['col'].mean())

In [ ]:
# Keep columns with at least 5 non-NA values
df = df.dropna(axis=1, thresh=5)

### Duplicate checking

In [ ]:
def check_duplicates(df, subset_columns=None):
    duplicate_report = {
        'exact_duplicates': df.duplicated().sum(),
        'partial_duplicates': df.duplicated(subset=subset_columns).sum() if subset_columns else 0
    }
    return duplicate_report

check_duplicates(df)

df.drop_duplicates(inplace = True)

# Remove duplicates based on subset of columns
df.drop_duplicates(subset=['col_a', 'col_b'], inplace =True)

### Outliers and distribution

In [ ]:
# Scatterplot for checking trends
sns.scatterplot(x=df['feature_a'], y=df['feature2'])
plt.title("Scatter Plot for Relationship Analysis")
plt.show()

In [ ]:
# Boxplots for all numerical features
numeric_df = df.select_dtypes(include='number')
n_cols = len(numeric_df.columns)

fig, axes = plt.subplots(n_cols, 1)

for i, col in enumerate(numeric_df.columns):
    sns.boxplot(x=numeric_df[col], ax=axes[i])
    axes[i].set_title(f"{col}")

plt.tight_layout()
plt.show()

In [ ]:
# Histogram for a numerical column
df['num_col'].hist(bins=20)
plt.title("Data Distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
# Log transformation
df['num_col'] = np.log1p(df['num_col'])

In [ ]:
# Filter outlier rows using IQR based rule
Q1 = df['num_col'].quantile(0.25)
Q3 = df['num_col'].quantile(0.75)
IQR = Q3 - Q1

# Filter out rows with outliers
df = df[(df['num_col'] >= Q1 - 1.5 * IQR) & (df['num_col'] <= Q3 + 1.5 * IQR)]

### Text cleaning

In [ ]:
# Trim spaces and convert to lowercase
df['column_str'] = df['colum_str'].str.strip().str.lower()

# Replace '&' with 'and'
df['column_str'] = df['colum_str'].str.replace('&', 'and')

# Remove punctuation
df['column_str'] = df['colum_str'].str.replace(r'[^\w\s]', '')

In [ ]:
# Categorize strings based on keyword. Example with colors

category_map = {
    'blue': ['AZURE', 'cerulean', 'sky blue'],
    'red': ['magenta', 'DARK RED', 'red orange']
}

def categorize_column(series, category_map):
    series = series.str.lower().fillna('') # clean up string formatting
    conditions = []
    choices = []

    for label, keywords in category_map.items():
        pattern = '|'.join(map(re.escape, keywords)) # use regex to escape special characters, create keyword search list
        conditions.append(series.str.contains(pattern, regex=True)) # check for keyword matches
        choices.append(label)

    return np.select(conditions, choices, default='other')

# Example usage
categorize_column(df['color'], category_map)    

In [ ]:
# Replace text value with contained keyword
place = df['Place']
name_a = place.str.contains('name_a')
name_b = place.str.contains('name_b')
df['Place'] = np.where(name_a, 'name_a',
                       np.where(name_b, 'name_b',
                                place.str.replace('-', '_')))

### Chain cleaning functions

In [ ]:
# Example of combining multiple cleaning functions together

def clean_data(df, category_map, text_col, new_col_name):
    # Drop duplicates
    df = df.drop_duplicates(subset=['col_a', 'col_b'])

    # Categorize column 
    df[new_col_name] = categorize_column(df[text_col], category_map)

    # Report missing values
    missing_report = check_missing_values(df)

    return df, missing_report

cleaned_df, missing_report = clean_data(df, category_map, 'color', 'color_category')    